<a href="https://colab.research.google.com/github/flying-bear/AutoT/blob/master/hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание 3. Разрешение семантической многозначности.

При выполнении домашнего задания можно пользоваться тетрадками с семинаров.

### Описание задания:

Выберите одну из предложенных лексем: _свет_, _острый_, _лук_, _замок_.

1. Выберите 2-3 значения лексемы по словарю (рекомендую использовать [МАС](http://feb-web.ru/feb/mas/mas-abc/), обязательно напишите, каким словарем пользовались).
Для каждого значения придумайте несколько "диагностических" слов (которые сигнализируют о реализации одного из значений в контексте): 
например, _ключ_ — _вода_ / _дверь_. Подберите по НКРЯ контексты для выбранной лексемы, содержащие диагностические слова.
Таким образом, получается размеченный тестсет.

2. Для каждого контекста определите значение автоматически — с помощью AdaGram / ELMo на выбор.
В случае с AdaGram вы сразу получаете какой-то номер значения, с ELMo — нужно кластеризовать.

3. Возьмите словарные толкования для каждого значения и припишите ему значение автоматически
тем же способом, что и в п.2.

4. Придумайте и реализуйте способ соотнести толкования с значениями с помощью тезауруса (Wiki_ru_wordnet / РуТез). 

5. Оцените accuracy — в какой доле контекстов автоматически определенно значение для контекста совпадает 
с автоматически определенным значением толкования.

### Критерии оценки:

по 2 балла на каждый пункт

### Формат сдачи задания:

Jupyter-notebook на гитхабе.

### Дедлайн: 

9 декабря 2019 10:00мск

## 1. Я выбрала слово _замок_.
МАС говорит о них следующее:
1. **за́мок**
  1. Дворец и крепость феодала. Средневековый замок. На скале видны развалины какого-то замка. Пушкин, Путешествие в Арзрум. || Большой помещичий дом затейливой архитектуры. С одной стороны Обломовка, с другой — княжеский замок, с широким раздольем барской жизни. И. Гончаров, Обломов.
  2. Устар. Здание тюрьмы. — Ослабели [арестанты], сидевши в замке, расслабли, а их ведут в самое пекло, — осуждал кого-то приказчик, обращаясь к подошедшему Нехлюдову. Л. Толстой, Воскресение. Пальтецо его вместе с мелочью в карманах так и осталось в тюремном замке. Федин, Необыкновенное лето.
2. **замо́к**
  1. Устройство для запирания чего-л. ключом. Дверной замок. Висячий замок. Запереть на замок. || Приспособление для смыкания, соединения концов чего-л. (цепочки, браслета и пр.).
  2. Воен. Приспособление для производства выстрела в некоторых видах огнестрельного оружия. Бойцы положили на траву, рядом с Пархоменко, пять пулеметов и замки с трех орудий. Вс. Иванов, Пархоменко.
  3. Тех. Способ скрепления бревен, брусьев и т. п. частей деревянных конструкций.
  4. Архит. Центральный камень в вершине свода, арки.

Я полагаю, что лучше всего будут классифицироваться наиболее частотные значения, а именно 1.1 - 'дворец' и 2.1 - 'устройство'.

Диагностические слова для значения 1.1: неприступный, король, фамильный

Диагностические слова для значения 2.1: ключ, дверь, щёлкнуть

In [0]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

### Скачаем размеченный датасет и положим в пандас

In [11]:
!wget https://raw.githubusercontent.com/flying-bear/AutoT/master/hw/3/homonyms.txt

--2019-12-07 22:06:37--  https://raw.githubusercontent.com/flying-bear/AutoT/master/hw/3/homonyms.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12331 (12K) [text/plain]
Saving to: ‘homonyms.txt’

homonyms.txt        100%[===================>]  12.04K  --.-KB/s    in 0s      

2019-12-07 22:06:37 (97.4 MB/s) - ‘homonyms.txt’ saved [12331/12331]



In [0]:
with open('homonyms.txt', 'r', encoding='utf-8') as f:
  lines = [line.strip().split('\t') for line in f.readlines()]

In [84]:
df = pd.DataFrame(lines, columns = ['sense', 'text']) 
df['sense'] = df['sense'].apply(int)
df['text'] = df['text'].apply(lambda s: s.strip())
df.head()

,sense,text
0,2,В этот момент щелкнул замок и дверь чуть приот...
1,2,"Мы поднялись, и я позвонил в дверь. Щелкнул за..."
2,2,"Андрей закрыл дверь, и никелированный замок гр..."
3,2,"Я зашёл в маленькую грязную комнату с лавкой, ..."
4,2,Дверь мягко щелкнула и ответила маслянистыми п...


##2. Я выбрала Адаграм

### Установим его

In [1]:
!wget https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib

--2019-12-07 21:59:56--  https://s3.amazonaws.com/kostia.lopuhin/all.a010.p10.d300.w5.m100.nonorm.slim.joblib
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.42.78
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.42.78|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1462416741 (1.4G) [application/x-www-form-urlencoded]
Saving to: ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’

all.a010.p10.d300.w 100%[===================>]   1.36G  16.3MB/s    in 87s     

2019-12-07 22:01:23 (16.0 MB/s) - ‘all.a010.p10.d300.w5.m100.nonorm.slim.joblib’ saved [1462416741/1462416741]



In [3]:
!pip install git+https://github.com/lopuhin/python-adagram.git

  Cloning https://github.com/lopuhin/python-adagram.git to /tmp/pip-req-build-m1xvfrrs
  Running command git clone -q https://github.com/lopuhin/python-adagram.git /tmp/pip-req-build-m1xvfrrs
  Created wheel for adagram: filename=adagram-0.0.1-cp36-cp36m-linux_x86_64.whl size=464608 sha256=0f23aadd7adc826bb099e6b4f896512f97d00c41c9482d8ced27096ea9f947c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-smfngs_j/wheels/11/0f/46/f5df96670df8f7973b4c2311ffc9b02e435a7bd3207f992c4d
Successfully built adagram


### Поставим всё остальное

In [2]:
!pip install pymystem3==0.1.10

In [14]:
!pip install wiki_ru_wordnet

### Imports

In [0]:
import adagram
import numpy as np
import re

from nltk.corpus import stopwords
from pymystem3 import Mystem

### Инициализация

In [0]:
mystem = Mystem()

In [0]:
vm = adagram.VectorModel.load('all.a010.p10.d300.w5.m100.nonorm.slim.joblib')

In [0]:
def preprocess(text): 
  """
  cleans a text of punctuation and case
  
  :param text: str to be cleaned
  :return: str in lowercase and stripped of punctuation
  """
  low = text.lower().strip()
  stripped = re.sub('!|\.|,|#|$|%|\\|\'|\(|\)|-|\+|\*|/|\:|;|<|>|=|\?|\[|\]|@|^|_|`|{|}|~', '', low)
  return stripped

def lemmatize(text):
  """
  preprocesses and lemmatizes a string in mystem

  :param text: str to be lemmatized
  :return: list of str, lemmas
  """
  return mystem.lemmatize(preprocess(text))

### Посмотрим на значения слова замок в Адаграме

In [53]:
for i, prob in vm.word_sense_probs('замок'):
  print(i+1)
  print(*vm.sense_neighbors('замок', i))

1
('орденский', 0, 0.6297175) ('монсегюр', 0, 0.615526) ('аббатство', 0, 0.6004314) ('дворец', 2, 0.5968682) ('сеньория', 0, 0.59168553) ('герцог', 2, 0.5900553) ('инстербург', 0, 0.57428056) ('каркассон', 0, 0.57125235) ('резиденция', 0, 0.5694655) ('амбуаз', 0, 0.56807244)
2
('задвижка', 0, 0.7730665) ('запор', 1, 0.74998415) ('засов', 1, 0.73436505) ('ключ', 0, 0.7206449) ('отмыкать', 0, 0.7102439) ('шпингалет', 0, 0.7052061) ('щеколда', 0, 0.70305103) ('защелка', 1, 0.70179313) ('запираться', 0, 0.696132) ('дверь', 6, 0.69068426)
3
('крепость', 1, 0.70243007) ('башня', 1, 0.6900965) ('бастион', 2, 0.66138613) ('средневековый', 1, 0.6602798) ('величественный', 1, 0.66022444) ('донжон', 0, 0.62887174) ('цитадель', 0, 0.62760615) ('башня', 3, 0.60489523) ('здание', 3, 0.6001993) ('крепостной', 0, 0.596294)
4
('крепость', 3, 0.60584474) ('дворец', 1, 0.5704742) ('башня', 5, 0.5693793) ('пещера', 0, 0.5673409) ('форт', 2, 0.5618624) ('холд', 1, 0.56155473) ('дом', 2, 0.5479651) ('подзем

Выглядит так, будто значения 1, 3 и 4 относятся к нашему значению 1 ('дворец'), а значение 2 - к нашему значению 2 ('устройство'). Будем иметь это ввиду, определяя точность позднее. (Сравним!)

In [0]:
senses = {1: {1, 3, 4}, 
          2 : {2}}

### Приступим к разрешению омонимии

In [0]:
def disambiguate(model, word, context):
  """
  disambiguates a word providing a number that
  corresponds to the meaning of a word in a context using adagram

  :param model: adagram model, pretrained
  :param word: str, word to be disambiguated
  :param contaext: str, context of the word
  :return: int, number corresponding to the meaning of the word in the context
  """
  word, _ = lemmatize(word)
  probs = model.disambiguate(word, lemmatize(context))
  return 1 + probs.argmax()

In [85]:
df['predict_sense_id'] = [disambiguate(vm, 'замок', context) for context in df['text']]
df.head()

,sense,text,predict_sense_id
0,2,В этот момент щелкнул замок и дверь чуть приот...,2
1,2,"Мы поднялись, и я позвонил в дверь. Щелкнул за...",2
2,2,"Андрей закрыл дверь, и никелированный замок гр...",2
3,2,"Я зашёл в маленькую грязную комнату с лавкой, ...",2
4,2,Дверь мягко щелкнула и ответила маслянистыми п...,2


## 3. Припишем номера значений словарным толкованиям

### Значение 1 'дворец'

In [57]:
def1 = 'Дворец и крепость феодала. Средневековый замок. На скале видны развалины какого-то замка. Пушкин, Путешествие в Арзрум. || Большой помещичий дом затейливой архитектуры. С одной стороны Обломовка, с другой — княжеский замок, с широким раздольем барской жизни. И. Гончаров, Обломов.'
sense_def1 = disambiguate(vm, 'замок', def1)
sense_def1

3

### Значение 2 'устройство'

In [58]:
def2 = 'Устройство для запирания чего-л. ключом. Дверной замок. Висячий замок. Запереть на замок. || Приспособление для смыкания, соединения концов чего-л. (цепочки, браслета и пр.).'
sense_def2 = disambiguate(vm, 'замок', def2)
sense_def2

2

### Запишем эти числа в датасет

In [86]:
df['auto_sense'] = df['sense'].apply(lambda x: 2 if x == 2 else 3)
df.head()

,sense,text,predict_sense_id,auto_sense
0,2,В этот момент щелкнул замок и дверь чуть приот...,2,2
1,2,"Мы поднялись, и я позвонил в дверь. Щелкнул за...",2,2
2,2,"Андрей закрыл дверь, и никелированный замок гр...",2,2
3,2,"Я зашёл в маленькую грязную комнату с лавкой, ...",2,2
4,2,Дверь мягко щелкнула и ответила маслянистыми п...,2,2


## 4. Соотнесём значения и толкования с помощью тезауруса

## 5. Подсчитаем точность

In [87]:
compare_strict = lambda dataframe: 1 if dataframe['auto_sense'] == dataframe['predict_sense_id'] else 0
df['compare_strict'] = df.apply(compare_strict, axis=1)
compare = lambda dataframe: 1 if  dataframe['predict_sense_id'] in senses[dataframe['sense']] else 0
df['compare'] = df.apply(compare, axis=1)
df.tail()

,sense,text,predict_sense_id,auto_sense,compare_strict,compare
55,1,"К примеру, привидение, обитающее в фамильном з...",1,3,0,1
56,1,"Удивительно, но только пару лет назад Ольга см...",1,3,0,1
57,1,"Он чокнулся с Астросом и Зарецким, поклонился ...",4,3,0,1
58,1,Я обнаружила в себе совершенно не свойственный...,4,3,0,1
59,1,"Продав все свои должности, он удалился в свой ...",1,3,0,1


Если считать строго, то значение, определённое автоматически неверно, однако значения 1, 3 и 4 в адаграме, судя по ближайшим к ним словам, очень близки, и compare отражает это.

In [92]:
accuracy_strict = np.mean(df['compare_strict'])
accuracy = np.mean(df['compare'])
print(f'strict:\t\t{accuracy_strict}\nnon-strict:\t{accuracy}')

strict:		0.55
non-strict:	0.9
